In [ ]:
from google.colab import drive

MOUNTPOINT = '/content/gdrive'
drive.mount(MOUNTPOINT)

In [ ]:
import pandas as pd
import tensorflow as tf

input_dataset = '/content/gdrive/My Drive/Colab Notebooks/datasets/ISIC_2019_Training_Input'
output_dataset = '/content/gdrive/My Drive/Colab Notebooks/datasets/ISIC_2019_Trainiing_GroundTruth.csv'

input_ds = tf.keras.utils.image_dataset_from_directory(directory=input_dataset, labels=None, shuffle=False, validation_subset=1)
output_df = pd.read_csv(output_dataset, names=['MEL', 'NV', 'BCC', 'AK', 'BKL', 'DF', 'VASC', 'SCC', 'UNK'])